In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = [224, 224]

train_path = 'data/train'
valid_path = 'data/test'

In [3]:

# add preprocessing layer to the front of VGG
vgg =  MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:

# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [5]:
folders = glob('data/train/*')

In [6]:
x = Flatten()(vgg.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)


In [8]:
model = Model(inputs=vgg.input, outputs=prediction)


In [9]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
from keras.preprocessing.image import ImageDataGenerator


In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4462 images belonging to 2 classes.


In [15]:

test_set = test_datagen.flow_from_directory('data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 911 images belonging to 2 classes.


In [16]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)



Epoch 1/10
140/140 [==============================] - 139s 993ms/step - loss: 0.3420 - accuracy: 0.9697 - val_loss: 3.3544e-04 - val_accuracy: 0.9989
Epoch 2/10
140/140 [==============================] - 139s 991ms/step - loss: 0.1850 - accuracy: 0.9870 - val_loss: 0.0000e+00 - val_accuracy: 0.9989
Epoch 3/10
140/140 [==============================] - 139s 991ms/step - loss: 0.1229 - accuracy: 0.9926 - val_loss: 0.0000e+00 - val_accuracy: 0.9912
Epoch 4/10
140/140 [==============================] - 140s 998ms/step - loss: 0.0954 - accuracy: 0.9944 - val_loss: 0.0000e+00 - val_accuracy: 0.9989
Epoch 5/10
140/140 [==============================] - 141s 1s/step - loss: 0.0461 - accuracy: 0.9971 - val_loss: 0.0000e+00 - val_accuracy: 0.9978
Epoch 6/10
140/140 [==============================] - 141s 1s/step - loss: 0.1089 - accuracy: 0.9944 - val_loss: 0.0000e+00 - val_accuracy: 0.9989
Epoch 7/10
140/140 [==============================] - 140s 1s/step - loss: 0.1874 - accuracy: 0.9908 - val

In [17]:
def save_model_to_db(model, client, db, dbconnection, model_name):
    import pickle
    import time
    import pymongo
    #pickling the model
    pickled_model = pickle.dumps(model)
    
    #saving model to mongoDB
    # creating connection
    myclient = pymongo.MongoClient(client)
    
    #creating database in mongodb
    mydb = myclient[db]
    
    #creating collection
    mycon = mydb[dbconnection]
    info = mycon.insert_one({model_name: pickled_model, 'name': model_name, 'created_time':time.time()})
    print(info.inserted_id, ' saved with this id successfully!')
    
    details = {
        'inserted_id':info.inserted_id,
        'model_name':model_name,
        'created_time':time.time()
    }
    print(details)
    
    return details

In [18]:
details = save_model_to_db(model = model, client ='mongodb://localhost:27017/', db = 'mydatabase_1', 
                 dbconnection = 'customers_1', model_name = 'cnn')

5eaae74888785bae831cd563  saved with this id successfully!
{'inserted_id': ObjectId('5eaae74888785bae831cd563'), 'model_name': 'cnn', 'created_time': 1588258632.9091914}


In [19]:
import tensorflow as tf

from keras.models import load_model

model.save('model_vgg19.h5')
